In [1]:
import numpy as np
import numpy.random as rnd
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from ipywidgets import widgets, interact, interactive, fixed, interact_manual

from IPython.display import display, clear_output
import string
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
alphabet = list(string.ascii_uppercase)
max_categories = len(alphabet)

In [3]:
# GUI elements
categories_slider = widgets.IntSlider(min=1, max=max_categories, description='Categories', continuous_update=False, value=4)
categories_slider.layout.width = 'auto'

uniform_button = widgets.Button(description='Uniform', tooltip='Set all sliders to a uniform distribution')
normal_button = widgets.ToggleButton(
    value=False,
    description='Normal',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Approximate a normal distribution',
    icon='times'
)
draw_button = widgets.Button(description='Draw', icon='fa-bar-chart', button_style='success')
sliders = []
texts = []

box = widgets.VBox()

In [4]:
std_slider = widgets.FloatSlider(min=0., max=5., step=0.1, value=0., continuous_update=False, description='Standard deviation')

In [5]:
out = widgets.Output()
out.layout.align_items = 'center'
fig = plt.figure()

In [6]:
def plot(x_axis, y_axis):
    with sns.axes_style("darkgrid"):
        fig.clear()
        ax = fig.add_subplot(1, 1, 1)
        ax.set_xlabel('Category')
        ax.set_ylabel('%')
        ax.set_ylim((0,100))
        sns.barplot(x=x_axis, y=y_axis, ax=ax, palette='pastel')
        fig.canvas.draw_idle()
    with out:
        clear_output()
        display(fig)

In [7]:
def cat_update(c_slider):
    categories = alphabet[0:categories_slider.value]
    probabilities = np.repeat(1./len(categories), categories_slider.value)
    
    plt.clf()
    plt.xlabel('Category')
    sns.barplot(x=pd.Series(categories), y=pd.Series(np.repeat(1, len(categories))), palette='Blues_d')
    plt.show()

In [8]:
def init():
    global sliders
    global box
    global texts
    global cats_per_x
    
    diff = categories_slider.value - len(sliders)

    if  diff > 0:
        for i in range(len(sliders), len(sliders)+diff):
            desc = alphabet[i]
            sliders.append(widgets.IntSlider(min=0, max=100, value=50, description=desc, continuous_update=False, orientation='vertical'))
            
            texts.append(widgets.IntText(
            value=1,
            description='1-'+str(categories_slider.value),
            disabled=False,
            max=len(sliders),
        ))
    elif diff < 0:
        sliders = sliders[0:categories_slider.value]
        texts = texts[0:categories_slider.value]
        
    cats_per_x = widgets.IntSlider(min=1, max=len(sliders), step=1)

    slider_box = widgets.HBox(sliders)
    dist_box = widgets.HBox([uniform_button, normal_button])
    box = widgets.VBox([dist_box, slider_box, draw_button, std_slider])
    
    
    pair_boxes = []
    #texts = []
    for i in range(1, len(sliders)+1):
        texts[i-1].layout.width='150px'
        vbox = widgets.VBox([sliders[i-1], texts[i-1]])
        vbox.layout.align_items='flex-end'
        pair_boxes.append(vbox)

    box = widgets.VBox([dist_box, widgets.HBox(pair_boxes), draw_button, std_slider])
    box.layout.align_items='center'
    
    normal_action(None)

def draw(change):
    init()
    # Sort according to value
    s = list(sliders)
    
    # Weights set, use that order, otherwise alphabetical
    if [t.value in range(1,len(sliders)) for t in texts]:
        #s=[s[t.value-1] for t in texts]
        zippy = zip([slider for slider in sliders], [text.value for text in texts])
        zippy.sort(key= lambda (slider,weight): weight)
        s=[slider for (slider,weight) in zippy]
    else:    
        s.sort(key=lambda x: x.description, reverse=False)
        
    s.sort(key=lambda x: x.value, reverse=True)
    
    probabilities = pd.Series([slider.value for slider in s])
    normalize_by = probabilities.sum()
    
    clear_output()
    display(box, out)
    
    plot(pd.Series([str(slider.description) for slider in s]), (probabilities/normalize_by)*100)
    
def uniform_action(change):
    
    for slider in sliders:
        slider.value=50
        

In [9]:
def normal_action(change):

    if(normal_button.value):
        for text in texts:
            normal_button.button_style = 'success'
            normal_button.icon = 'check'
            text.disabled = False
            text.layout.visibility = 'visible'
    
    else:
        for text in texts:
            normal_button.button_style = 'danger'
            normal_button.icon = 'times'
            text.disabled = True
            text.layout.visibility = 'hidden'
        

In [10]:
interact(cat_update, c_slider=categories_slider);

interactive(children=(IntSlider(value=4, continuous_update=False, description=u'Categories', layout=Layout(wid…

In [11]:
def normal_dist(change):
    
    if change == 'value':
        
        if std_slider.value == 0.:
            sliders[0].value = 100
            for slider in sliders[1:]:
                slider.value = 0
        else:
            spacing = list(np.arange(start=0., stop=len(sliders), step=1.))
            values = stats.norm.cdf(spacing,loc=0, scale=std_slider.value)
            skew_by_one = np.concatenate([[0.], values[0:len(values)-1]])
            diff = values-skew_by_one

            for slider in sliders:
                
                index = sliders.index(slider)
                # If all IntTexts are within the correct range, use that order
                if [t.value in range(1,len(sliders)) for t in texts]:
                    index = texts[sliders.index(slider)].value-1
                
                slider.value = int(np.round(diff[index]*100, 0))

In [12]:
init()

draw_button.on_click(draw)
uniform_button.on_click(uniform_action)
normal_button.on_trait_change(normal_action)

std_slider.on_trait_change(normal_dist)

display(box, out)

Output(layout=Layout(align_items=u'center'))